
# 10 — Unified RAG + Agents + MCP Starter Template

A reusable starter notebook for:
- Loading documents
- Building a vector store
- Defining a base Agent
- Wiring a simple LangGraph workflow
- Stubbing an MCP-style tool registry

You can copy this into new projects and adapt.



## 1. Basic Project Configuration


In [ ]:

import os
from dataclasses import dataclass

@dataclass
class RAGConfig:
    data_path: str = "./data"
    db_path: str = "./chroma_db"
    model: str = "gpt-4o-mini"

cfg = RAGConfig()
cfg



## 2. Loader, Chunker, Embedder, VectorStore Manager


In [ ]:

!pip install langchain langchain-openai chromadb langchain-community langgraph --quiet

import os
from typing import List
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

os.makedirs(cfg.data_path, exist_ok=True)
with open(os.path.join(cfg.data_path, "sample.txt"), "w") as f:
    f.write("This is a starter RAG project template with LangChain and LangGraph.\n")
    f.write("You can replace this text with your own domain documents.\n")

class DataLoader:
    def __init__(self, path: str):
        self.path = path

    def load(self):
        loader = TextLoader(self.path)
        return loader.load()

class Chunker:
    def __init__(self, chunk_size=400, overlap=40):
        self.splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)

    def split(self, docs):
        return self.splitter.split_documents(docs)

class VectorStoreManager:
    def __init__(self, persist_dir: str):
        self.persist_dir = persist_dir
        self.emb = OpenAIEmbeddings()
        os.makedirs(self.persist_dir, exist_ok=True)

    def build(self, chunks):
        self.db = Chroma.from_documents(chunks, self.emb, persist_directory=self.persist_dir)
        return self.db

    def as_retriever(self, k=4):
        return self.db.as_retriever(search_kwargs={"k": k})

data_loader = DataLoader(os.path.join(cfg.data_path, "sample.txt"))
docs = data_loader.load()
chunker = Chunker()
chunks = chunker.split(docs)
vsm = VectorStoreManager(cfg.db_path)
db = vsm.build(chunks)
retriever = vsm.as_retriever()



## 3. Base Agent and Simple RAG Agent


In [ ]:

from abc import ABC, abstractmethod

llm = ChatOpenAI(model=cfg.model)

class BaseAgent(ABC):
    @abstractmethod
    def run(self, **kwargs):
        ...

class RAGAgent(BaseAgent):
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm

    def run(self, question: str) -> str:
        docs = self.retriever.get_relevant_documents(question)
        ctx = "\n".join(d.page_content for d in docs)
        prompt = f"Context:\n{ctx}\n\nQuestion: {question}\nAnswer:"
        return self.llm.invoke(prompt).content

rag_agent = RAGAgent(retriever, llm)
print(rag_agent.run(question="What is this project template about?"))



## 4. Simple LangGraph Workflow (Planner → RAG Agent)


In [ ]:

from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain_core.messages import HumanMessage

class RAGState(TypedDict):
    question: str
    plan: str
    answer: str

def planner_node(state: RAGState) -> RAGState:
    prompt = f"Create a short search plan for answering: {state['question']}"
    out = llm.invoke([HumanMessage(content=prompt)])
    state["plan"] = out.content
    return state

def rag_node(state: RAGState) -> RAGState:
    # For now, ignore plan and just call RAGAgent
    state["answer"] = rag_agent.run(question=state["question"])
    return state

graph = StateGraph(RAGState)
graph.add_node("planner", planner_node)
graph.add_node("rag", rag_node)
graph.set_entry_point("planner")
graph.add_edge("planner", "rag")
graph.add_edge("rag", END)

workflow = graph.compile()

result = workflow.invoke({"question": "How can I extend this RAG template?"})
result



## 5. MCP-Style Tool Registry (Stub)


In [ ]:

class ToolRegistry:
    def __init__(self):
        self.tools = {}

    def register(self, name: str, fn):
        self.tools[name] = fn

    def call(self, name: str, **kwargs):
        if name not in self.tools:
            raise ValueError(f"Tool {name} not found")
        return self.tools[name](**kwargs)

tool_registry = ToolRegistry()

def example_search_tool(query: str) -> str:
    return f"[Search results placeholder for: {query}]"

tool_registry.register("web_search", example_search_tool)

print(tool_registry.call("web_search", query="RAG architectures"))



## End of Notebook 10  
This starter template is ready to be copied into new RAG + Agents + MCP projects.
